### This script allows to get better control over tasks in earth engine

* Purpose of script: allows the user to list tasks and cancel all pending tasks
* Author: Rutger Hofste
* Kernel used: python27
* Date created: 20170913

In [1]:
import time
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
print(dateString,timeString)

('Y2018M04D17', 'UTC 12:10')


# Settings

In [2]:
MAXTASKS = 10
CANCELTASKS = 0

In [3]:
import pandas as pd
import ee
from retrying import retry
import datetime
import random

In [4]:
ee.Initialize()

# Functions

In [5]:
def get_tasks():
    return ee.batch.Task.list()

def cancel_task(task):
    print task
    random_time = random.random()
    time.sleep(0.5+random_time*0.5)
    if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY') :
        print 'canceling %s' % task
        task.cancel()
        
        
@retry(wait_exponential_multiplier=10000, wait_exponential_max=100000)
def checkStatus(task):
    return ee.batch.Task.status(task)
           
def get_details(taskList,MAXTASKS):
    df = pd.DataFrame()
    for i in range(0,min(len(taskList),MAXTASKS)):
        dictNew = checkStatus(taskList[i])
        dfNew = pd.DataFrame(dictNew, index=[i])
        try:
            dfNew["calctime(min)"] = (dfNew["update_timestamp_ms"]-dfNew["start_timestamp_ms"])/(1000*60)
            dfNew["queuetime(min)"] = (dfNew["start_timestamp_ms"]-dfNew["creation_timestamp_ms"])/(1000*60)
            dfNew["runtime(min)"]= dfNew["queuetime(min)"]+dfNew["calctime(min)"]
            dfNew["start_timestamp_UTC"] = datetime.datetime.fromtimestamp(dfNew["start_timestamp_ms"]/1000).strftime('%H:%M:%S')
        except:
            pass
        df = df.append(dfNew)
        print i
    return df
    


In [6]:
taskList = get_tasks()


In [7]:
detailedTasks = get_details(taskList,MAXTASKS)

0
1
2
3
4
5
6
7
8
9


In [8]:
detailedTasks

,calctime(min),creation_timestamp_ms,description,id,output_url,progress,queuetime(min),runtime(min),start_timestamp_UTC,start_timestamp_ms,state,task_type,update_timestamp_ms
0,0.004550,1523967059312,Asset ingestion: projects/WRI-Aquaduct/PCRGlob...,7CCGQ4JNR227KI2QIZUUXVNV,NaN,0.0,0.094267,0.098817,12:11:04,1523967064968,RUNNING,INGEST,1523967065241
1,0.003133,1523967056103,Asset ingestion: projects/WRI-Aquaduct/PCRGlob...,LYMV7R5SFX6F2PQY3RLWEOOJ,NaN,0.0,0.088233,0.091367,12:11:01,1523967061397,RUNNING,INGEST,1523967061585
2,0.004517,1523967052885,Asset ingestion: projects/WRI-Aquaduct/PCRGlob...,6MNZPQHTMYNE2T3RLYOWLSAY,NaN,0.0,0.068950,0.073467,12:10:57,1523967057022,RUNNING,INGEST,1523967057293
3,0.004500,1523967049542,Asset ingestion: projects/WRI-Aquaduct/PCRGlob...,RKUL4JCQWD4FCSPJGHVC6F53,NaN,0.0,0.051883,0.056383,12:10:52,1523967052655,RUNNING,INGEST,1523967052925
4,0.004533,1523967046365,Asset ingestion: projects/WRI-Aquaduct/PCRGlob...,A7FOQ73BBBACIDYV7DYHHPUD,NaN,0.0,0.046367,0.050900,12:10:49,1523967049147,RUNNING,INGEST,1523967049419
5,0.003133,1523967043192,Asset ingestion: projects/WRI-Aquaduct/PCRGlob...,LOCKVQRR3FYGVJDNCVZ4PUGC,NaN,0.0,0.090850,0.093983,12:10:48,1523967048643,RUNNING,INGEST,1523967048831
6,0.004500,1523967040034,Asset ingestion: projects/WRI-Aquaduct/PCRGlob...,I7RFZUC6WUHRRRYSQZAGNMPB,NaN,0.0,0.085133,0.089633,12:10:45,1523967045142,RUNNING,INGEST,1523967045412
7,0.004483,1523967036849,Asset ingestion: projects/WRI-Aquaduct/PCRGlob...,G2MKVN23IS4PABY5HTXR6AB7,NaN,0.0,0.064800,0.069283,12:10:40,1523967040737,RUNNING,INGEST,1523967041006
8,0.004533,1523967033639,Asset ingestion: projects/WRI-Aquaduct/PCRGlob...,UJ53IAYH52YEGTPGGONHZPC4,NaN,0.0,0.108983,0.113517,12:10:40,1523967040178,RUNNING,INGEST,1523967040450
9,0.546083,1523967030408,Asset ingestion: projects/WRI-Aquaduct/PCRGlob...,3Y4GZRQMVHFLTWCGTGVJ332K,https://code.earthengine.google.com/?asset=pro...,NaN,0.080117,0.626200,12:10:35,1523967035215,COMPLETED,INGEST,1523967067980


# DANGER ZONE

In [9]:
if CANCELTASKS == 1:
    pendingTasks = [task for task in taskList if task.config['state'] in (u'RUNNING',u'UNSUBMITTED',u'READY')]
    for task in pendingTasks:
        cancel_task(task)